In [1]:
# Importing Libraries
import import_ipynb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from importlib import reload

import datetime

import NI_M1
import NI_M2
reload(NI_M1)
reload(NI_M2)


importing Jupyter notebook from NI_M1.ipynb
importing Jupyter notebook from NI_M2.ipynb
importing Jupyter notebook from NI_M1.ipynb
importing Jupyter notebook from NI_M2.ipynb


<module 'NI_M2' from 'NI_M2.ipynb'>

In [2]:

# Import dataset
ReadCSV = False
if ReadCSV:
    df_ini = pd.read_csv(r'O:\GIRFT DiRAC\FlavienHardy\2_NosocomialInfections\src\0.processing\data\NI_Final.csv')


# Elective admissions: to do again with update AllEpisodes

In [159]:

df = df_ini

FilterAdmission = (df['POD_EL'] + df_ini['POD_DC'] >= 1)
Filter_LOS_ElAdm = (df['Spell_Los']>7) & FilterElectiveAdm
df['UP_LOS_ElAdm'] = np.where(Filter_LOS_ElAdm, 1, 0)

print('Number of infections identified:', df['UP_LOS_ElAdm'].sum())
ColumnList = ['POD_EL', 'POD_DC', 'Spell_Los', 'UP_LOS_ElAdm']

df5 = df[df['UP_LOS_ElAdm']==1][ColumnList]

print('------------------------------------')
print('These infections correspond to ELECTIVE admissions, with LOS > 7')
df5.head(2)


Number of infections identified: 2523
------------------------------------
These infections correspond to ELECTIVE admissions, with LOS > 7


,POD_EL,POD_DC,Spell_Los,UP_LOS_ElAdm
20,1,0,34,1
50,1,0,24,1


# Emergency admissions

In [42]:

# Import dataset
ReadCSV = False
if ReadCSV:
    df_AllEpisodes = pd.read_excel(r'O:\GIRFT DiRAC\FlavienHardy\2_NosocomialInfections\Data\0_Raw\MarchData\All episodes 07-06-21.xlsx')


### Find first episodes

In [ ]:

df2 = df_AllEpisodes

# Format date features
DateFeatures = ['Admission_date', 'EPIstart']
for feature in DateFeatures:
    df2[feature] = pd.to_datetime(df2[feature])

# Find first episode with Covid-19
Columnslist = ['P_Spell_ID', 'Admission_date', 'EPIstart']
IndexFirstEpisode = df2[Columnslist].groupby(by=['P_Spell_ID', 'Admission_date'])['EPIstart'].idxmin().values
df2 = df2.iloc[IndexFirstEpisode]
df2.head(2)


In [ ]:

# Keep infections diagnosed AFTER admission
Filter = ( pd.to_datetime(df2['EPIstart']) - pd.to_datetime(df2['Admission_date']) > datetime.timedelta(days=0))
df3 = df2[Filter]
df3.head(2)


In [161]:

# Keep columns of interest
ColumnList = ['P_Spell_ID', 'HESID', 'EPIstart', 'Admission_date']
df3_Sub = df3[ColumnList]

print('These Spell_IDs correspond to spells where the FIRST occurence of Covid-19 appeared AFTER the start of the spell.')
df3_Sub.head(2)


These Spell_IDs correspond to spells where the FIRST occurence of Covid-19 appeared AFTER the start of the spell.


,P_Spell_ID,HESID,EPIstart,Admission_date
23776,1656.0,000092DDD60956DDAD79301E546F2D9B,2020-03-25,2020-03-24
100003,12041.0,00040034499D14709B5E6E441D0C612C,2020-04-10,2020-04-01


### Link with POD dataset

In [101]:

# Import dataset
ReadCSV = True
if ReadCSV:
    df_Assembled = pd.read_csv(r'O:\GIRFT DiRAC\FlavienHardy\2_NosocomialInfections\src\0.processing\data\NI_Assembled.csv')


C:\Users\flavien.hardy\OneDrive - NHS Improvement\Desktop\FH_Local\env\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [163]:

ColumnsList = ['P_Spell_ID', 'POD', 'Spell_Los']
df_Assembled_Sub = df_Assembled[ColumnsList]

Filter_LOS_EM = (df_Assembled_Sub['Spell_Los']>7) & (df_Assembled_Sub['POD']=='EM')
List_FirstEpisode = df3_Sub['P_Spell_ID'].values
FilterFirstEpisode = (df_Assembled_Sub['P_Spell_ID'].isin(List_FirstEpisode))
Filter_EM = (Filter_LOS_EM) & (FilterFirstEpisode)

df_4 = df_Assembled_Sub[ Filter_EM ]
print('Number of infections identified:', df_4.shape)
print('------------------------------------')
print('These spells correspond to EM admissions with LOS > 7 days, and first infection recorded AFTER start of spell.')
df_4.head(2)

Number of infections identified: (39746, 3)
------------------------------------
These spells correspond to EM admissions with LOS > 7 days, and first infection recorded AFTER start of spell.


,P_Spell_ID,POD,Spell_Los
0,1487000,EM,54
4,3445944,EM,55
